In [120]:
import pandas as pd
import requests

# Function to call in Reddit info via API

**Below: This function:**

1) Variabilizes the 'base' url of the Pushshift API

2) Uses two editable dictionaries of parameters to add-on to the API URL
    - Current parameters are `subreddit` and `size`.

3) Assign the `HTTP Reponse` request and variabilize the `status code`.

4) An `if` statement checks that the value of the `status_code` for both subreddits are within the bounds of `Success 2XX`.

5) If successful:
    - The `JSON` is read-in for each subreddit, 
    - The posts are saved from each JSON, 
    - And the posts are cast into a Pandas DataFrame

In [141]:
def generate_dataframes(subreddit_str_1, subreddit_str_2, size_1, size_2):
    
    # Setup URL of API
    base_url = "https://api.pushshift.io/reddit/search/submission"    
    
    # Create the params of the API URL
    params_1 = {
        "subreddit": subreddit_str_1,
        "size": size_1
    }
    params_2 = {
        "subreddit": subreddit_str_2,
        "size": size_2
    }
    
    # Response
    res_1 = requests.get(base_url, params_1)
    res_2 = requests.get(base_url, params_2)
    res_check_1 = res_1.status_code
    res_check_2 = res_2.status_code
    
    # Check response is good
    if (res_check_1 >= 200 and res_check_1 < 300) and (res_check_2 >= 200 and res_check_2 < 300):
        
        # Create DataFrame:
        data_1 = res_1.json()
        data_2 = res_2.json()
        
        posts_1 = data_1["data"]
        posts_2 = data_2["data"]
        
        df_1 = pd.DataFrame(posts_1)
        df_2 = pd.DataFrame(posts_2)
        return df_1, df_2
    else:
        return f"Check HTTP: {res_check1, res_check_2}"

In [139]:
lpt_df, ulpt_df = generate_dataframes("LifeProTips", "UnethicalLifeProTips", 100, 100)

In [20]:
df[["subreddit", "title", "selftext"]].head()  # Title has 'LPT' in it, could throw the model off 

,subreddit,title,selftext
0,LifeProTips,LPT: need an expensive prescription drug but h...,
1,LifeProTips,"LPT: After a shower, dry yourself in there to ...","So, I personally don't know if doing that is j..."
2,LifeProTips,"LPT: After a shower, dry up in there to keep y...",[deleted]
3,LifeProTips,If shady people come up and ask you if your na...,NaN
4,LifeProTips,LPT: If you’re going to bring a birthday cake ...,
